In [3]:
import pandas as pd
import os

In [2]:
## read data in

In [55]:
def split_coor_basin(basin_file):
    ''' split coordinate for each basin row data
    '''
    basin = pd.read_excel(basin_file)
    basin.set_index('Unnamed: 0',inplace=True)
    basin_row_con = {} ## data container
#     basin_row_2d = [] ## basin row container
    #iterate rows
    for i in range(1,len(basin)):
        basin_row_list = [] # row data container
        basin_row = basin.iloc[i]
        for j in range(len(basin_row)):
            basin_row_pari = []
            if ',' in str(basin_row[j]) and basin_row[j]:
                basin_row_col_split = str(basin_row[j]).split(',') #split data
                basin_row_pari.append(basin_row_col_split[0]) #saving long
                basin_row_pari.append(basin_row_col_split[1])
            basin_row_list.append(basin_row_pari) #saving pair to basin_row
        basin_row_con[basin_row.name] = basin_row_list ## append list to 2d row container
#         basin_row_con[basin_row.name] = basin_row_2d
    return basin_row_con

In [56]:
split_basin_data = split_coor_basin('basin.xlsx')

In [144]:
def get_lat_long_range(split_basin_data):
    '''for each basin df columns, split data to basin name, lat, long 
    '''
    basins_range = [] ##basin coordinate range container
#     lat_min = 0
#     lat_max = 0
#     long_min = 0
#     long_max = 0
#     basin_df_range = pd.DataFrame()
    basin_df = pd.DataFrame(split_basin_data)
    for col in basin_df.columns: # for each dataframe columns

        basin_range = {}
        basin_range['basin_name'] = col
#         basin_col = basin_df[col] 
        basin_col_long = []
        basin_col_lat = []
        for row_data in basin_df[col]: # for each row in basin_col
#             basin_df_range['basin_name'] = col
            if row_data and len(row_data) == 2 : #and '-' not in row_data
#                 print(row_data)
#                 print(type(row_data))
#                 row_data = row_data[0] ## get lis value
#                 print(row_data)
                basin_col_long.append(row_data[0])
                basin_col_lat.append(row_data[1])
#         print(basin_col_long)
#         print(basin_col_lat)
#         break
        ## get basin lat and long range
        basin_range['long_min'] = min(basin_col_long) 
        basin_range['long_max'] = max(basin_col_long)
        basin_range['lat_min'] = min(basin_col_lat)
        basin_range['lat_max'] = max(basin_col_lat)
        
        basins_range.append(basin_range)
        
    return basins_range
        
#                 basin_df_range['lat'] = basin_col[i].split(',')[1]  #basin lat   

In [162]:
basins_range = get_lat_long_range(split_basin_data)

In [221]:
basins_range[1]
basins_range_df = pd.DataFrame.from_records(basins_range)

In [222]:
basins_range_df.head()

,basin_name,long_min,long_max,lat_min,lat_max
0,Faroe - Shetland Escarpment,-0.007939999999962311,1.912470000000041,60.08393867900003,64.33822000000004
1,Porcupine,-11.28532999999993,-13.98371999999995,49.14216000000004,53.27784000000002
2,North Lewis,-5.176649999999937,-7.00010999999995,58.28707000000003,59.75718721400006
3,Blake Plateau Ultradeep,-75.47845999999998,-76.85803,27.28387000000003,31.01669322200006
4,Doukkala Ultradeep,-10.00062999999994,-9.980449999999962,30.29466081500004,36.64070000000003


In [148]:
field = pd.read_excel('field.xlsx')
field_arr = field[['field','long','lat']]
field_arr.set_index('field',inplace=True)

In [156]:
field_arr.iloc[1]

long    66.0
lat     37.0
Name: SHIBARGHAN, dtype: float64

In [239]:
def get_basin(field_file,basins_range):
    '''project field lat and long to correspoding basin name
    
    Args:
        field,dataframe with storing field coordinates
        basin,basin coordinates
    Returns:
        field-basin, corresponding dataframe
        
    '''
    ## read in data
    field = pd.read_excel(field_file)
#     basin = pd.read_excel('basin.xlsx')
    field_basin = {}
    field_arr = field[['field','long','lat']]
    field_arr.set_index('field',inplace=True)
    field_arr['basin'] = None
    for i in range(len(field_arr)): #for each row 
        row_data = field_arr.iloc[i] 
        long_lat_data = row_data.to_list()
        long = long_lat_data[0]  #field long
        lat = long_lat_data[1]   #field lat
#         print(long,lat)
#         print(long.values())
#         print(type(long))
        for basin_range in basins_range: ## get basin name
            if str(long) >= basin_range['long_min'] \
                and str(long) <= basin_range['long_max'] \
                and str(lat) >= basin_range['lat_min']\
                and str(lat) <= basin_range['lat_max']:
#                 print(basin_range['basin_name'])
        
#                 field_basin[row_data.name] = basin_range['basin_name'] #get basin name
                field_arr['basin'][i] = basin_range['basin_name']
#                 print('find one corresponding basin')
    return field_arr    
#     return field_basin
            
    
    

In [240]:
field_basin = get_basin('field.xlsx',basins_range)

D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for la